In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob

In [2]:
fnames = glob('./csv/*.csv')
print (fnames)

['./csv\\9.35333_bigru_batt_max_focal.csv', './csv\\9.36666_bahdanau_attentions_max_focal.csv', './csv\\9.36666_bigru_batt_max_bce.csv', './csv\\9.37333_bilstm_batt_max_focal.csv', './csv\\9.39333_bigru_batt_max_focal.csv', './csv\\9.40000_bahdanau_norm_attentions_max_focal.csv', './csv\\9.40000_bigru_batt_mean_focal.csv', './csv\\9.40666_bigru_batt4_max_focal.csv', './csv\\9.40666_deep_bigru_batt_max_focal.csv', './csv\\9.41333_bigru_batt_drop_max_focal.csv', './csv\\9.41333_bigru_lnatt_max_focal.csv', './csv\\9.42000_bahdanau_attentions_max, cat4.csv', './csv\\9.42666_bigru_bnatt_max_focal.csv', './csv\\9.43333_bigru_batt_5_max_focal.csv']


In [3]:
NUM_OPTIONS = 100
TOP_AT = 10

In [4]:
# Cache all predictions to numpy array.
num_files = len(fnames)
num_rows = pd.read_csv(fnames[0])['Id'].values.shape[0]
preds = np.zeros((num_files, num_rows, NUM_OPTIONS)).astype(np.uint8)
for f, fname in enumerate(fnames):
    Predict = pd.read_csv(fname)['Predict'].values.astype(str)
    for r, str_labels in enumerate(Predict):
        labels = str_labels.split('-')[:NUM_OPTIONS]
        for c, label in enumerate(labels):
            if np.isnan(float(label)):
                continue
            preds[f, r, c] = int(label)

In [5]:
sum_preds = np.sum(preds, axis=0)
print (sum_preds.shape)

(1000, 100)


In [6]:
best_ids = np.argsort(sum_preds, axis=-1)

In [7]:
# Create submit
submit = pd.read_csv(fnames[0])

ensembled_labels = []
for r, name in enumerate(submit['Id']):
    ensembled_label = np.zeros(NUM_OPTIONS).astype(np.uint8)
    ensembled_label[best_ids[r, -TOP_AT:]] = 1
    ensembled_labels.append(''.join(['1-' if x else '0-' for x in ensembled_label]))
submit['Predict'] = ensembled_labels
submit.to_csv('Ensemble.csv', index=False)